In [2]:
import pandas as pd
import numpy as np
from pyvttbl import DataFrame
from pyvttbl.stats import Anova

##### SET STUDY VARIABLES
## Between-factors >> group on....
# Dep: Exptertise, FATR: Study
human = True
human = False

FATR = False
FATR = True


if FATR:
    print("** FATR Study **")
    source = os.path.join("data","fatr_students.csv")
    bfactor = "Study"
else:
    print("** Depend Study **")
    source = os.path.join("data","depend_only.csv")
    bfactor = "Expertise"


temp_csv_path = 'temp_cleaned_data.csv'

## Set DVs >> process multiple in one script, or just one at a time; 
dv_cols = ['Diagnosis Time']  
dv_cols = ['Cooper Harper']  
dv_cols = ['NasaTLX']
dv_cols = ['Decision']


# Create a pyvttbl.DataFrame from the cleaned pandas DataFrame
df = DataFrame()
df_pandas = pd.read_csv(source)

# Calculate NasaTLX if this is the DV for analysis
if dv_cols[0] != "NasaTLX":
    ## Drop rows with missing values in relevant columns
    # CAUTION! If dv_cols list contains multiple columns, then columns missing row values will obliterate other columns' row values
    df_pandas = df_pandas.dropna(subset=dv_cols)
    temp_csv_path = 'temp_cleaned_data.csv'
    df_pandas.to_csv(temp_csv_path, index=False)
else:
    print("create NasaTLX composit measure...")
    # Invert 'Performance' scores
    df_pandas['Performance'] = 11 - df_pandas['Performance']
    columns = ['Performance', 'Temporal Demand', 'Frustration Level', 'Mental Demand', 'Effort']
    for column in columns:
        median = df_pandas[column].median()
        df_pandas[column].fillna(median)
    # Calculate the NasaTLX composite score
    df_pandas['NasaTLX'] = df_pandas[columns].mean(axis=1)
    df_pandas.to_csv(temp_csv_path, index=False)

df.read_tbl(temp_csv_path)

def print_aov_and_marginal_means():
    aov = {'A': 1, 'B': 2}  # Example dictionary
    marginal_means = [3, 4]  # Example list
    print("AOV:", aov)
    print("Marginal Means:", marginal_means)
    
for dv in dv_cols:
    print(f"Running ANOVA for: {dv}")
    
    aov = Anova()
    aov.run(df, dv, wfactors=['Scenario', 'Fault'], bfactors=[bfactor])   
    aov.truncate(test='gg')

    if human:
        ### HUMAN READABLE
        print(aov)
        print("\n\n") # this puts spaces between reports when running multiples
    else:
        ### For the Robots
        print(dict(aov))
        print("\n\n") 
    
        marginal_means = aov.get_marginal_means(['Scenario', 'Fault', bfactor])
        print(f"Marginal Means for: {dv}")
        print(marginal_means)
        print("\n\n") 

yes fatr
Running ANOVA for: Decision
('Scenario',)
('Study',)
('Fault', 'Study')
('Scenario', 'Fault', 'Study')
{('Fault',): {'y2': array([0.66666667, 0.83333333]), 'df': 1.0, 'eps_gg': 1.0, 'eps_hf': 1.0, 'eps_lb': 1.0, 'ss': 2.666666666666666, 'mss': 2.666666666666666, 'dfe': 22.0, 'sse': 4.791666666666667, 'mse': 0.2178030303030303, 'F': 12.243478260869562, 'p': 0.0020286288906186467, 'eta': 0.17391304347826084, 'obs': 48.0, 'critT': 2.073873067904015, 'se': 0.07127503522131787, 'ci': 0.139699069033783, 'lambda': 26.713043478260865, 'power': 0.9985217320274492, 'df_gg': 1.0, 'dfe_gg': 22.0, 'mss_gg': 2.666666666666666, 'mse_gg': 0.2178030303030303, 'F_gg': 12.243478260869562, 'p_gg': 0.0020286288906186467, 'obs_gg': 48.0, 'critT_gg': 2.073873067904015, 'se_gg': 0.07127503522131787, 'ci_gg': 0.139699069033783, 'lambda_gg': 26.713043478260865, 'power_gg': 0.9985217320274492, 'df_hf': 1.0, 'dfe_hf': 22.0, 'mss_hf': 2.666666666666666, 'mse_hf': 0.2178030303030303, 'F_hf': 12.24347826086

In [16]:
import anthropic
from IPython.display import display, Markdown
api_key = os.getenv('ANTHROPIC_API_KEY')
client = anthropic.Anthropic()

model_switch = 0

model_map = {
    0: "claude-instant-1.2",
    1: "claude-3-opus-20240229",
}

model = model_map.get(model_switch, "claude-instant-1.2")
message = client.messages.create(
    model=model,
    max_tokens=1000,
    temperature=0,
    system="You will be provided with two pieces of information:\n\n    A dictionary containing the results of an ANOVA, including F-values, p-values, effect sizes, and other relevant statistics for main effects, interactions, and other sources of variance.\n    A dictionary containing the estimated marginal means and standard errors for each factor and their combinations.\n\nYour task is to create a well-structured and formatted LaTeX report that interprets the ANOVA results and includes the estimated marginal means and standard errors. The report should follow this outline:\n\n    If the data contains results which are not significant, then disregard that variable. The statistical significance score for p=0.05 determines what needs to be reported.\n\n    Begin with a brief introductory statement about the effects on the dependent variable.\n    Report the main effects, including:\n        F-values, p-values, and effect sizes for each main effect\n        Interpretation of the estimated marginal means and standard errors for each level of the main effects\n    Report the two-way interactions, including:\n        F-values, p-values, and effect sizes for each two-way interaction\n        Interpretation of the estimated marginal means and standard errors for each combination of the interacting factors\n    Report the three-way interaction, if present, including:\n        F-value, p-value, and effect size for the three-way interaction\n        Interpretation of the estimated marginal means and standard errors for each combination of the interacting factors\n    Conclude with a summary statement highlighting the key findings and the complex interplay among the factors.\n\nUse the following LaTeX formatting guidelines:\n\n    Use the description environment for the overall structure\n    Use paragraph for the main sections (e.g., Main effects, Two-way interactions)\n    Use itemize for individual findings within each section\n    Use appropriate LaTeX commands for mathematical symbols and equations (e.g., $F(1, 22) = 15.30$, $p < .001$, $\\eta^2 = .13$)\n    Indent the LaTeX code for better readability and structure\n\nProvide the complete LaTeX code, without preamble and document environment. The output should be formatted in a raw markdown panel for easy copy.",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Running ANOVA for: Decision ('Scenario',) ('Study',) ('Fault', 'Study') ('Scenario', 'Fault', 'Study') {('Fault',): {'y2': array([0.66666667, 0.83333333]), 'df': 1.0, 'eps_gg': 1.0, 'eps_hf': 1.0, 'eps_lb': 1.0, 'ss': 2.666666666666666, 'mss': 2.666666666666666, 'dfe': 22.0, 'sse': 4.791666666666667, 'mse': 0.2178030303030303, 'F': 12.243478260869562, 'p': 0.0020286288906186467, 'eta': 0.17391304347826084, 'obs': 48.0, 'critT': 2.073873067904015, 'se': 0.07127503522131787, 'ci': 0.139699069033783, 'lambda': 26.713043478260865, 'power': 0.9985217320274492, 'df_gg': 1.0, 'dfe_gg': 22.0, 'mss_gg': 2.666666666666666, 'mse_gg': 0.2178030303030303, 'F_gg': 12.243478260869562, 'p_gg': 0.0020286288906186467, 'obs_gg': 48.0, 'critT_gg': 2.073873067904015, 'se_gg': 0.07127503522131787, 'ci_gg': 0.139699069033783, 'lambda_gg': 26.713043478260865, 'power_gg': 0.9985217320274492, 'df_hf': 1.0, 'dfe_hf': 22.0, 'mss_hf': 2.666666666666666, 'mse_hf': 0.2178030303030303, 'F_hf': 12.243478260869562, 'p_hf': 0.0020286288906186467, 'obs_hf': 48.0, 'critT_hf': 2.073873067904015, 'se_hf': 0.07127503522131787, 'ci_hf': 0.139699069033783, 'lambda_hf': 26.713043478260865, 'power_hf': 0.9985217320274492, 'df_lb': 1.0, 'dfe_lb': 22.0, 'mss_lb': 2.666666666666666, 'mse_lb': 0.2178030303030303, 'F_lb': 12.243478260869562, 'p_lb': 0.0020286288906186467, 'obs_lb': 48.0, 'critT_lb': 2.073873067904015, 'se_lb': 0.07127503522131787, 'ci_lb': 0.139699069033783, 'lambda_lb': 26.713043478260865, 'power_lb': 0.9985217320274492}, ('Scenario', 'Fault'): {'y2': array([0.75 , 0.8125 , 0.58333333, 0.85416667]), 'df': 1.0, 'eps_gg': 1.0, 'eps_hf': 1.0, 'eps_lb': 1.0, 'ss': 1.0416666666666665, 'mss': 1.0416666666666665, 'dfe': 22.0, 'sse': 3.291666666666668, 'mse': 0.14962121212121218, 'F': 6.9620253164556924, 'p': 0.01500386028216144, 'eta': 0.07598784194528874, 'obs': 24.0, 'critT': 2.073873067904015, 'se': 0.0835443460494988, 'ci': 0.16374691825701765, 'lambda': 7.594936708860756, 'power': 0.7498528512708454, 'df_gg': 1.0, 'dfe_gg': 22.0, 'mss_gg': 1.0416666666666665, 'mse_gg': 0.14962121212121218, 'F_gg': 6.9620253164556924, 'p_gg': 0.01500386028216144, 'obs_gg': 24.0, 'critT_gg': 2.073873067904015, 'se_gg': 0.0835443460494988, 'ci_gg': 0.16374691825701765, 'lambda_gg': 7.594936708860756, 'power_gg': 0.7498528512708454, 'df_hf': 1.0, 'dfe_hf': 22.0, 'mss_hf': 1.0416666666666665, 'mse_hf': 0.14962121212121218, 'F_hf': 6.9620253164556924, 'p_hf': 0.01500386028216144, 'obs_hf': 24.0, 'critT_hf': 2.073873067904015, 'se_hf': 0.0835443460494988, 'ci_hf': 0.16374691825701765, 'lambda_hf': 7.594936708860756, 'power_hf': 0.7498528512708454, 'df_lb': 1.0, 'dfe_lb': 22.0, 'mss_lb': 1.0416666666666665, 'mse_lb': 0.14962121212121218, 'F_lb': 6.9620253164556924, 'p_lb': 0.01500386028216144, 'obs_lb': 24.0, 'critT_lb': 2.073873067904015, 'se_lb': 0.0835443460494988, 'ci_lb': 0.16374691825701765, 'lambda_lb': 7.594936708860756, 'power_lb': 0.7498528512708454}, ('Scenario', 'Study'): {'y2': array([0.85416667, 0.70833333, 0.70833333, 0.72916667]), 'df': 1.0, 'eps_gg': 1.0, 'eps_hf': 1.0, 'eps_lb': 1.0, 'ss': 0.6666666666666666, 'mss': 0.6666666666666666, 'dfe': 22.0, 'sse': 2.4583333333333335, 'mse': 0.11174242424242425, 'F': 5.966101694915253, 'p': 0.023076638811863698, 'eta': 0.049999999999999996, 'obs': 24.0, 'critT': 2.073873067904015, 'se': 0.0721987243175518, 'ci': 0.1415094996624015, 'lambda': 6.508474576271187, 'power': 0.6839216064969531, 'df_gg': 1.0, 'dfe_gg': 22.0, 'mss_gg': 0.6666666666666666, 'mse_gg': 0.11174242424242425, 'F_gg': 5.966101694915253, 'p_gg': 0.023076638811863698, 'obs_gg': 24.0, 'critT_gg': 2.073873067904015, 'se_gg': 0.0721987243175518, 'ci_gg': 0.1415094996624015, 'lambda_gg': 6.508474576271187, 'power_gg': 0.6839216064969531, 'df_hf': 1.0, 'dfe_hf': 22.0, 'mss_hf': 0.6666666666666666, 'mse_hf': 0.11174242424242425, 'F_hf': 5.966101694915253, 'p_hf': 0.023076638811863698, 'obs_hf': 24.0, 'critT_hf': 2.073873067904015, 'se_hf': 0.0721987243175518, 'ci_hf': 0.1415094996624015, 'lambda_hf': 6.508474576271187, 'power_hf': 0.6839216064969531, 'df_lb': 1.0, 'dfe_lb': 22.0, 'mss_lb': 0.6666666666666666, 'mse_lb': 0.11174242424242425, 'F_lb': 5.966101694915253, 'p_lb': 0.023076638811863698, 'obs_lb': 24.0, 'critT_lb': 2.073873067904015, 'se_lb': 0.0721987243175518, 'ci_lb': 0.1415094996624015, 'lambda_lb': 6.508474576271187, 'power_lb': 0.6839216064969531}, ('SUBJECT',): {'ss': 2.5, 'sse': 2.125, 'mse': 0.09659090909090909, 'df': 23.0, 'dfe': 22.0}, ('TOTAL',): {'ss': 18.0, 'df': 95.0}, ('WITHIN',): {'ss': 15.5, 'df': 72.0}, ('Scenario', 'SUBJECT'): {'ss': 2.4583333333333335, 'df': 22.0, 'mss': 0.11174242424242425}, ('Fault', 'SUBJECT'): {'ss': 4.791666666666667, 'df': 22.0, 'mss': 0.2178030303030303}, ('Scenario', 'Fault', 'SUBJECT'): {'ss': 3.291666666666668, 'df': 22.0, 'mss': 0.14962121212121218}} Marginal Means for: Decision ({'Scenario': array([0.8125, 0.6875]), 'Fault': array([0.58333333, 0.91666667]), 'Study': array([0.8125, 0.6875]), 'ScenarioFault': array([0.75 , 0.875 , 0.41666667, 0.95833333]), 'ScenarioStudy': array([0.95833333, 0.66666667, 0.66666667, 0.70833333]), 'FaultStudy': array([0.625 , 0.54166667, 1. , 0.83333333]), 'ScenarioFaultStudy': array([0.91666667, 0.58333333, 1. , 0.75 , 0.33333333, 0.5 , 1. , 0.91666667])}, {'Scenario': array([0.05693291, 0.06761023]), 'Fault': array([0.07191241, 0.04031495]), 'Study': array([0.05693291, 0.06761023]), 'ScenarioFault': array([0.09028939, 0.06895966, 0.10279899, 0.04166667]), 'ScenarioStudy': array([0.04166667, 0.09829464, 0.09829464, 0.09477599]), 'FaultStudy': array([0.10094661, 0.10389457, 0. , 0.07770873]), 'ScenarioFaultStudy': array([0.08333333, 0.1486471 , 0. , 0.13055824, 0.14213381, 0.15075567, 0. , 0.08333333])}, {'Scenario': [('LOFW',), ('SGTR',)], 'Fault': [('Spoof',), ('True Fault',)], 'Study': [('Dependency',), ('FATR',)], 'ScenarioFault': [('LOFW', 'Spoof'), ('LOFW', 'True Fault'), ('SGTR', 'Spoof'), ('SGTR', 'True Fault')], 'ScenarioStudy': [('LOFW', 'Dependency'), ('LOFW', 'FATR'), ('SGTR', 'Dependency'), ('SGTR', 'FATR')], 'FaultStudy': [('Spoof', 'Dependency'), ('Spoof', 'FATR'), ('True Fault', 'Dependency'), ('True Fault', 'FATR')], 'ScenarioFaultStudy': [('LOFW', 'Spoof', 'Dependency'), ('LOFW', 'Spoof', 'FATR'), ('LOFW', 'True Fault', 'Dependency'), ('LOFW', 'True Fault', 'FATR'), ('SGTR', 'Spoof', 'Dependency'), ('SGTR', 'Spoof', 'FATR'), ('SGTR', 'True Fault', 'Dependency'), ('SGTR', 'True Fault', 'FATR')]})"
                }
            ]
        }
    ]
)



# HTML template for the code block
print(f'Results from Claude {model} for LaTeX')

html_template = f"""
<div style="background-color: #f6f8fa; padding: 10px; border-left: 6px solid #ccc; font-family: monospace; overflow: auto;">
<pre><code style="color: #333;">{message_formatted}</code></pre>
</div>
"""

display(HTML(html_template))



SyntaxError: unterminated string literal (detected at line 48) (349201866.py, line 48)

In [12]:
from IPython.display import display, HTML

code = """
# Some Python code
def hello_world():
    print("Hello, world!")
"""

# HTML template for the code block
html_template = f"""
<div style="background-color: #f6f8fa; padding: 10px; border-left: 6px solid #ccc; font-family: monospace; overflow: auto;">
<pre><code style="color: #333;">{code}</code></pre>
</div>
"""

display(HTML(html_template))
